<small><i>Actualizado a 11/04/25 - Este cuaderno Jupyter pertenece a Alberto Galgo Jiménez (agalgo@naturgy.com). </i></small>

<div class  = "alert alert-success"><b>FILTRADO DE BASE DE DATOS FORMACIÓN</b><p>

### <span style="color:blue"><em>Importación Paquetes</em></span>

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import simpledialog, messagebox

### <span style="color:blue"><em>Funciones con TKINTER</em></span>

#### <span style="color:red"><em>Importación del archivo indicando ruta y nombre</em></span>

In [11]:
def preguntar_ruta_archivo():
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal
    ruta_archivo = simpledialog.askstring("Importar archivo", "Ingresa la ruta del archivo Excel que deseas importar:")
    nombre_archivo = simpledialog.askstring("Nombre del archivo", "Ingresa el nombre del archivo (con extensión .xlsx):")
    return f"{ruta_archivo}/{nombre_archivo}"

#### <span style="color:red"><em>Definir campos a filtrar</em></span>

In [12]:
def preguntar_columnas(df):
    columnas = df.columns.tolist()
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal
    respuesta = simpledialog.askstring("Columnas", f"Columnas disponibles: {', '.join(columnas)}\nSelecciona máximo 3 columnas para filtrar (separadas por comas):")
    columnas_seleccionadas = respuesta.split(',')
    return [columna.strip() for columna in columnas_seleccionadas]

#### <span style="color:red"><em>Elección de valores de filtrado</em></span>

In [13]:
def preguntar_filtros(df, columnas_interes):
    filtros = {}
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal
    
    # Preguntar solo por las columnas especificadas
    for columna in columnas_interes:
        opciones = df[columna].unique()
        respuesta = simpledialog.askstring("Filtros", f"Opciones para {columna}: {', '.join(map(str, opciones))}\nSelecciona las opciones para {columna} (puedes usar operadores como >, <, >=, <=, !=, seleccionar varios valores separados por comas o deja en blanco para no aplicar filtro):")
        if respuesta:  # Si la respuesta no está vacía
            # Convertir a tipo adecuado y almacenar la condición
            if df[columna].dtype == 'int64' or df[columna].dtype == 'float64':
                filtros[columna] = respuesta
            else:
                filtros[columna] = respuesta.split(',')
        root.update #actualiza la ventana principal.

    root.destroy # desaparece la ventana Con esto conseguimos subsanar el error que da TKINTER de rendimiento
    
    return filtros

#### <span style="color:red"><em>Aplicación de los filtros</em></span>

In [14]:
def aplicar_filtros(df, filtros):
    df_filtrado = df.copy()
    for columna, condicion in filtros.items():
        try:
            if df[columna].dtype in ['int64', 'float64']:
                # Evaluar la condición numérica
                df_filtrado = df_filtrado.query(f"{columna} {condicion}")
            else:
                # Evaluar la condición de texto
                if isinstance(condicion, list):
                    if '!=' in condicion[0]:
                        opciones = [opcion.strip() for opcion in condicion[0].replace('!=', '').split(',')]
                        df_filtrado = df_filtrado[~df_filtrado[columna].isin(opciones)]
                    else:
                        opciones = [opcion.strip() for opcion in condicion]
                        df_filtrado = df_filtrado[df_filtrado[columna].isin(opciones)]
                else:
                    if '!=' in condicion:
                        opciones = [opcion.strip() for opcion in condicion.replace('!=', '').split(',')]
                        df_filtrado = df_filtrado[~df_filtrado[columna].isin(opciones)]
                    else:
                        df_filtrado = df_filtrado[df_filtrado[columna] == condicion]
        except Exception as e:
            print(f"Error al aplicar el filtro '{condicion}' en la columna '{columna}': {e}")
    
    return df_filtrado



#### <span style="color:red"><em>Exportacion df</em></span>

In [15]:
def preguntar_ruta_guardado():
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal
    ruta = simpledialog.askstring("Guardar archivo", "Ingresa la ruta donde deseas guardar el archivo Excel:")
    nombre_archivo = simpledialog.askstring("Nombre del archivo", "Ingresa el nombre del archivo (sin extensión):")
    return ruta, nombre_archivo

### <span style="color:blue"><em>Importación y tratamiento de datos</em></span>

In [16]:
ruta_archivo = preguntar_ruta_archivo()

In [17]:
# Importar el archivo Excel
df = pd.read_excel(ruta_archivo)
df

,ID_personal,Edad,Antigüedad,Ant_puesto,dependientes,formacion,excluido,dpo,eval,directivo
0,12287,65,14,6,0,Ingeniero,0,0.00,0.00,No directivo
1,12286,48,18,6,0,Ingeniero,1,0.79,2.58,No directivo
2,44408080,43,15,7,0,Ingeniero,1,1.00,2.77,No directivo
3,22572,52,19,7,0,Ingeniero,1,0.99,2.68,No directivo
4,12010,54,17,7,0,Master,1,1.13,3.02,No directivo
...,...,...,...,...,...,...,...,...,...,...
5902,ARGas-33,51,23,1,1,Ingeniero,1,1.05,2.60,No directivo
5903,ARGas-534,47,23,1,1,Ingeniero,1,1.03,2.22,No directivo
5904,ARGas-M11190,34,7,1,1,Ingeniero,1,0.89,3.67,No directivo
5905,ARGas-12029,45,17,1,1,Master,1,1.12,3.83,No directivo


### <span style="color:blue"><em>Ejecución filtro 1 (max.3 filtros)</em></span>

In [18]:
# Preguntar al usuario por las columnas a filtrar
columnas_interes = preguntar_columnas(df)

# Preguntar al usuario por los filtros
filtros = preguntar_filtros(df, columnas_interes)

# Aplicar los filtros al dataframe
df_filtrado = aplicar_filtros(df, filtros)
df_filtrado

,ID_personal,Edad,Antigüedad,Ant_puesto,dependientes,formacion,excluido,dpo,eval,directivo
41,70116,47,19,5,18,Master,1,1.19,3.67,Directivo
102,44300550,49,23,2,13,Ingeniero,1,1.15,4.05,Directivo
178,44182801,58,33,2,11,Ingeniero,1,0.86,1.15,No directivo
190,44110051,51,22,5,21,Master,1,0.98,2.86,Directivo
222,44972025,47,2,2,18,Master,1,1.16,3.40,Directivo
...,...,...,...,...,...,...,...,...,...,...
5468,50523,47,18,3,16,Master,1,1.17,4.98,Directivo
5656,12015,52,16,2,11,Master,1,1.00,4.55,Directivo
5734,44391150,51,24,5,25,Master,1,0.98,4.56,Directivo
5834,11382,57,33,7,11,Ingeniero,1,1.15,2.83,Directivo


### <span style="color:blue"><em>Bucle siguiente filtro (max.3 filtros)</em></span>

In [19]:
while True:
    respuesta = messagebox.askquestion("Filtrar", "¿Quieres aplicar más filtros? (Max. 3)")
    if respuesta == "no":
        ruta_guardado, nombre_archivo = preguntar_ruta_guardado()
        ruta_completa = f"{ruta_guardado}/{nombre_archivo}.xlsx"
        df_filtrado.to_excel(ruta_completa, index=False)
        messagebox.showinfo("Exportado", f"DataFrame exportado a {ruta_completa}")
        break  # Salir del bucle después de exportar
    elif respuesta == "yes":
        columnas_interes = preguntar_columnas(df_filtrado)
        filtros = preguntar_filtros(df_filtrado, columnas_interes)
        df_filtrado = aplicar_filtros(df_filtrado, filtros)  # Actualizar df_filtrado en lugar de crear df_filtrado2
